# Crowd dynamics simulation — a multi-agent system based on CA

# Tasks

## Neighbourhood
Fill in the missing part of the *\_\_init\_\_* method in the *Board* class. Prepare two versions: one with Moore and one with von Neumann neighbourhood. Do not initialise neighbourhoods for border cells.
## Static potential field (2 points)

### Implement method *calculateField* in *Board* class:
* Create a list of points (*toCheck*) for which the static field should be recalculated (at the beginning all *staticFields* are equal to 100000). 
    * For all exits (points with *pointType == 2*) change the *staticField* to 0. 
    * Add all exits' neighbours to the *toCheck* list.
* Until the list *toCheck* is empty:
    * chech whether the first element of the list changed its *staticField* (call *calcStaticField*).
    * If so, add all its neighbours to the *toCheck* list.
    * Remove the first element from the list.

### Implement method *calcStaticField* in *Point* class:
* Find the smallest *staticField* in the cell's neighbourhood.
* If the cell's *staticField* is greater than the found value + 1, change the *staticField* to the found value + 1 and return *True*, otherwise do not change anything and return *False*.

### Implement *randomBoard* method:
* Add some walls and pedestrians, add at least one exit.
* Try to do this randomly.

## Naive Implementation ( 2 points)

### Implement *move* method:
* If the cell represents a pedestrian (*pointType == 3*), move the pedestrian to the neighbouring cell with the lowest *staticField*.

Run the simulation and observe what is happening. What's wrong? Try to find why some artefacts can be observed.

## Improvements (2 points)
Two main reasons responsible for the errors are:
* No "exit" mechanism. After reaching the exit, the agent should be removed.
* No cells synchronisation. One should note that agent moving down and right can reach the destination in one iteration.
* To fix this issue add a boolean value *isBlocked = false* to the *Point* class. After the agent moves, the *isBlocked* value of the occupied cell should change to *True*. Remember to "unblock" all cells at the beginning of each iteration. 

Correct the errors and create a complex board with walls, many pedestrians and at least two exits.




In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
from IPython.display import HTML

In [10]:
class Point:
    def __init__(self):
        self.neighbours = []
    # 0 - floor, 1 - wall, 2 - exit, 3 - pedestrian
        self.pointType = 0
        self.staticField = 100000
        self.isBlocked = False

    def clear(self):
        self.staticField = 100000


    def calcStaticField(self):
        # TODO
        minimum = 100000
        for point in self.neighbours:
            if point.staticField < minimum:
                minimum = point.staticField
            
        if self.staticField > minimum + 1:
            self.staticField = minimum + 1
            return True
        else:
            return False

    def move(self):
        # TODO
        if self.pointType == 3 and not self.isBlocked:
            minimum = 100000
            minimum_idx = 0
            i = 0
            for i, point in enumerate(self.neighbours):
                # check if wall
                if point.staticField < minimum and point.pointType != 1:
                    minimum = point.staticField
                    minimum_idx = i
            # check if exit
            if self.neighbours[minimum_idx].pointType == 2:
                pass
            else:
                # assign
                self.neighbours[minimum_idx].pointType = 3
                self.neighbours[minimum_idx].isBlocked = True
            
            self.pointType = 0
        
        return


    def getColour(self):
        if self.pointType == 0:
            # Floor
            return self.staticField*3
        elif self.pointType == 1:
            # Wall
            return 255
        elif self.pointType == 2:
            # Exit
            return 64
        else:
            # Pedestrian
            return 128
        


In [11]:
    def addNeighboursToList(point, toCheck):
        for neighbour in point.neighbours:
            toCheck.append(neighbour)



In [12]:
class Board:
    
    def __init__(self, xSize, ySize):
        self.points = []
        for i in range(0, xSize):
            row = []
            for j in range(0, ySize):
                row.append(Point())
            self.points.append(row)
            
        self.randomBoard()
                
        for i in range(1, xSize-1):
            for j in range(1, ySize-1):
                # TODO initialize NEUMANN neighbourhood here
                # Moore
                tmpList = [self.points[i-1][j-1], self.points[i-1][j], self.points[i-1][j+1],
                          self.points[i][j-1], self.points[i][j+1], self.points[i+1][j-1], self.points[i+1][j], self.points[i+1][j+1]]
    
                self.points[i][j].neighbours = tmpList
                                                    
    def randomBoard(self):
        xSize = len(self.points)
        ySize = len(self.points[0])
        # door
        # self.points[1][random.randint(1,ySize-2)].pointType = 2
        # self.points[1][random.randint(1,ySize-2)].pointType = 2
        self.points[random.randint(1,xSize-2)][random.randint(1,ySize-2)].pointType = 2
        self.points[random.randint(1,xSize-2)][random.randint(1,ySize-2)].pointType = 2
        # pedestrians
        self.points[1][1].pointType = 3
        for i in range(20):
            self.points[random.randint(1,xSize-2)][random.randint(1,ySize-2)].pointType = 3
        # walls
        for i in range(6):
            x = random.randint(1,xSize-2)
            y = random.randint(1,ySize-2)
            self.points[x][y].pointType = 1
            if x != xSize-1 and y != ySize-1:
                if random.random()>0.8:
                    self.points[x+1][y].pointType = 1
                if random.random()>0.8:
                    self.points[x][y+1].pointType = 1        
        
        return

        
    def iteration(self):  
        # unblock all cels
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].isBlocked = False
        # move
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].move()

    
    def clear(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].clear()
        self.calculateField()
    
    def redraw(self):
        # Returns numpy array of colours
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].getColour()
        return fig
    
    def getFieldMap(self):
        # Returns numpy array of fieldValues
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].staticField
        return fig
    
    def calculateField(self):
        # TODO 
        toCheck = []
        
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                if self.points[i][j].pointType == 2:
                    self.points[i][j].staticField = 0
                    addNeighboursToList(self.points[i][j], toCheck)
                    
        while toCheck:
            if toCheck[0].calcStaticField() == True:
                addNeighboursToList(toCheck[0], toCheck)
            toCheck.pop(0)
                
        return
            

In [13]:
def update(frameNum, board, im):
    if(frameNum == 0):
        im.set_data(board.redraw()) 
    else:   
        board.iteration()
        im.set_data(board.redraw()) 
    return im,

In [14]:
%%capture
# Parameters
steps = 100
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [17]:
#Run Simulation
import random 

board = Board(30,30)

board.clear()

im = ax.imshow(board.redraw(),  vmin=0, vmax=255)
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, im])
HTML(ani.to_jshtml())
